In [1]:
import googlemaps
import geopy.distance
from pymongo import MongoClient
from datetime import datetime
import os
import requests
import json
import time
import pandas as pd
import folium
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
##function to connect to MOngoDB
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

##Function to calculate distance between two points
def getLocCoord(office):
    longitude = office['longitude']
    latitude = office['latitude']
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

##Search places from coordinates
def nearbysearchName(query,location,radius):
    API_key = os.getenv('key')
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    places = []
    params = {
        'rankbydistance':"",
        'location': location,
        'radius': radius,
        'name': query,
        'key': API_key
    }
    res = requests.get(endpoint_url, params = params)
    results =  json.loads(res.content)
    places.extend(results['results'])
    return places

def nearbysearchText(query,location,radius):
    API_key = os.getenv('key')
    endpoint_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    places = []
    params = {
        'location': location,
        'radius': radius,
        'query': query,
        'key': API_key
    }
    res = requests.get(endpoint_url, params = params)
    results =  json.loads(res.content)
    places.extend(results['results'])
    return places


## get location from places
def getLocation(result):
    location = []
    for i in range(len(result)):
        latitude = result[i]['geometry']['location']['lat']
        longitude = result[i]['geometry']['location']['lng']
        name = result[i]['name']
        address = result[i]['vicinity']
        loc = {
            'name': name,
            'address': address,
            'longitude': longitude,
            'latitude': latitude
        }
        location.append(loc)
    return location

##Function to find closest airport
def closest_airport(office,df):
    coords_off = (office.get('latitude'),office.get('longitude'))
    dist = []
    for i in range(len(df)):
        coords_air = (df.iloc[i].get('lat'),df.iloc[i].get('lon'))
        dist.append(geopy.distance.geodesic(coords_off, coords_air).km)
    return min(dist)


In [3]:
airports=pd.read_csv("./output/filtered_airports.csv")
offices=pd.read_csv("./output/filtered_offices.csv")

In [4]:
airports.iloc[0]

Unnamed: 0                                        0
code                                            ABQ
lat                                         35.0494
lon                                        -106.625
name              Albuquerque International Airport
city                                    Albuquerque
state                                    New Mexico
country                               United States
woeid                                      12518564
icao                                           KABQ
direct_flights                                   41
carriers                                         25
Name: 0, dtype: object

In [6]:
print(len(offices),len(airports))
display(offices)

85 50


,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies
0,0,Thoof,web,Austin,USA,30.268735,-97.745209,0,36
1,1,RockYou,games_video,Redwood City,USA,37.484619,-122.206893,1,23
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26
3,3,oDesk,web,Redwood City,USA,37.479952,-122.178557,0,23
4,4,Bazaarvoice,software,Austin,USA,30.407545,-97.717667,0,36
...,...,...,...,...,...,...,...,...,...
80,80,Virdia,cleantech,Redwood City,USA,37.523311,-122.259154,0,23
81,81,Buzzient,enterprise,Boston,USA,42.362856,-71.083762,0,26
82,82,Nuventix,cleantech,Austin,USA,30.239229,-97.834211,0,36
83,83,Pearl Therapeutics,biotech,Redwood City,USA,37.502169,-122.217727,0,23


In [7]:
print(offices.iloc[0])
print(airports.iloc[0])
print(closest_airport(offices.iloc[0],airports))

Unnamed: 0               0
name                 Thoof
category               web
city                Austin
country                USA
latitude           30.2687
longitude         -97.7452
is_gaming                0
total_companies         36
Name: 0, dtype: object
Unnamed: 0                                        0
code                                            ABQ
lat                                         35.0494
lon                                        -106.625
name              Albuquerque International Airport
city                                    Albuquerque
state                                    New Mexico
country                               United States
woeid                                      12518564
icao                                           KABQ
direct_flights                                   41
carriers                                         25
Name: 0, dtype: object
10.618504347609518


In [8]:
dist_airport=[]
for x in range(len(offices)):
    office=offices.iloc[x]
    dist_airport.append(closest_airport(office,airports))
print(len(dist_airport))

85


In [9]:
offices['dist_airport'] = dist_airport 

In [10]:
display(offices.head())

,Unnamed: 0,name,category,city,country,latitude,longitude,is_gaming,total_companies,dist_airport
0,0,Thoof,web,Austin,USA,30.268735,-97.745209,0,36,10.618504
1,1,RockYou,games_video,Redwood City,USA,37.484619,-122.206893,1,23,21.824782
2,2,Carbonite,network_hosting,Boston,USA,42.350571,-71.076869,0,26,4.652931
3,3,oDesk,web,Redwood City,USA,37.479952,-122.178557,0,23,24.078317
4,4,Bazaarvoice,software,Austin,USA,30.407545,-97.717667,0,36,23.260582


In [14]:
starbucks =pd.DataFrame(getLocation(nearbysearchName('starbucks','30.268735,-97.745209',250))).iloc[0]

print(starbucks)

name                                   Starbucks
address      600 Congress Ave Ste. G-270, Austin
longitude                                -97.743
latitude                                 30.2683
Name: 0, dtype: object


In [ ]:
##Function to find closest airport
def closest_starbucks(office):
    coords_off = (office.get('latitude'),office.get('longitude'))
    coords_off2 = ("{},{}".format(coords_off[0],coords_off[1])
    starbucks =pd.DataFrame(getLocation(nearbysearchName("starbucks",coords_off2,5000))).iloc[0]:
    coords_str=(df.iloc[i].get('lat'),df.iloc[i].get('lon'))
    dist = []
    dist.append(geopy.distance.geodesic(coords_off, coords_str).km)
            
    
    for i in range(len(df)):
        coords_air = (df.iloc[i].get('lat'),df.iloc[i].get('lon'))
        dist.append(geopy.distance.geodesic(coords_off, coords_air).km)
    return min(dist)

dist_starbucks=[]
for x in range(len(offices)):
    office=offices.iloc[x]
    dist_airport.append(closest_airport(office,airports))
print(len(dist_airport))